<a href="https://colab.research.google.com/github/mechaturtles/SOST4Website/blob/master/Hypertension_%26_CVD_Risk_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hypertension and CVD Risk Prediction by Forrest Dipert

**Instructions:** 

Scroll down the page and click the arrow "play button" as you scroll past blocks of code to intialize and run each block. The lower blocks of code must be ran after the earlier blocks as they are dependent on eachother.

**Inputs:**

This function utilizes manually reported data like blood pressure, cholesterol, and other metrics which are accessible to patients in a telehealth format. Then employs published and peer reviewed models for predicting the onset of hypertension and cardiovascular disease to provide both patients and providers with trustworthy estimates of how their present health may affect their future health.

**Output:**

Prediction of hypertension onset is applied to non-hypertensive patients and have a one year outlook.

Prediciton of CVD onset to pre-hypertensive and hypertensive patients and has a 10 year outlook.

A patient profile of a 67 year old male (prehypertensive, non-smoking, non-diabetic) is provided below. If further test profiles are warranted change the PID value from 1 to 2 or 3.




Main Dict - Patient Directory
  
  Sub Dict - PID_X [1,2,3]
    
    Keys:

      Age [yrs],
      Date ["month/day/year"], 
      Sex [0 = Female, 1 = Male], 
      Hist_Hyp [ 0 = none, 1 = one parent, 2 = two parent], 
      Diag_Hyp [ 0 = none, 1 = pre-hyp, 2 = hyp], 
      Smoking [0 = no, 1 = yes], 
      Height [meters], 
      Weight [kg], 
      SBP [mmHg], 
      DBP [mmHg], 
      HDL [mg/dL], 
      Cholesterol [mg/dL]

 Main Dict - Hypertension Risk Points
  Keys:

    SBP [ mmHg, point],
    Sex [ 0/1, point],
    BMI [ kg/m^2, point],
    Age_DBP matrix of DBP first row & age in the first column,
    Smoking [0/1, point],
    Hist_Hyp [0/1/2, point],
    Risk_1yr [risk points, percent risk]

CVDBetas = [Age, Cholesterol, HDL, nontreated SBP, treated SBP, smoker, diabetes] floats

CVD_RiskNorm = CVD risk for a normal person

In [ ]:
PatientDirectory = dict(
    PID_1 = dict(
            Age = [67, 67, 67],
            Date = ["1/7/2020", "2/7/2020", "3/7/2020"], 
            Sex = [1, 1, 1],
            Hist_Hyp = [0, 0, 0],
            Diag_Hyp = [1, 1, 1],
            Diabetic = [0, 0, 0],
            Smoking = [0, 0, 0],
            Height = [1.7, 1.7, 1.7],
            Weight = [70, 72, 69],
            SBP = [130, 145, 135],
            DBP = [ 83, 77, 84],
            HDL = [44, 51, 46],
            Cholesterol = [ 161, 180, 201],
            BMI = [0, 0, 0], #CalcedByFunction
            Risk_Hyp = [0, 0, 0 ], #CalcedByFunction
            Risk_CVD = [0, 0, 0 ] #CalcedByFunction
            ),
    PID_2 = dict(
            Age = [61, 61, 61],
            Date = ["1/7/2020", "2/7/2020", "3/7/2020"],  
            Sex = [0, 0, 0],
            Hist_Hyp = [1, 1, 1],
            Diag_Hyp = [2, 2, 2],
            Diabetic = [0, 0, 0],
            Smoking = [1, 1, 1],
            Height = [1.7, 1.7, 1.7],
            Weight = [70, 72, 69],
            SBP = [130, 145, 135],
            DBP = [ 83, 77, 84],
            HDL = [47, 51, 46],
            Cholesterol = [ 180, 170, 201],
            BMI = [0, 0, 0], #CalcedByFunction
            Risk_Hyp = [0, 0, 0 ], #CalcedByFunction
            Risk_CVD = [0, 0, 0 ] #CalcedByFunction
            ),
    PID_3 = dict(
            Age = [53, 53, 53],
            Date = ["1/7/2020", "2/7/2020", "3/7/2020"],  
            Sex = [1, 1, 1],
            Hist_Hyp = [2, 2, 2],
            Diag_Hyp = [2, 2, 2],
            Diabetic = [1, 1, 1],
            Smoking = [0, 0, 0],
            Height = [1.7, 1.7, 1.7],
            Weight = [70, 72, 69],
            SBP = [125, 135, 145],
            DBP = [ 83, 77, 84],
            HDL = [55, 51, 46],
            Cholesterol = [ 161, 170, 201],
            BMI = [0, 0, 0], #CalcedByFunction
            Risk_Hyp = [0, 0, 0 ], #CalcedByFunction
            Risk_CVD = [0, 0, 0 ] #CalcedByFunction
            ),
    )

#The Hypertension risk point dictionary provides a set of matrices comprised of parameter values and their correlating points for identifying risk of developing hypertension
HypertensionRiskPoints = dict(
    SBP = [[0, -4],
           [110, 0], 
           [115, 2], 
           [120, 4], 
           [125, 6], 
           [134, 8], 
           [135, 10]],
    Sex = [[0, 1], 
           [1, 0]],
    BMI = [[0, 0], 
           [25, 1], 
           [30, 3]],
    Age_DBP = [[ 999, 0, 70, 75, 80, 85], 
               [20, -8, -3, 0, 3, 6],
               [30, -5, 0, 2, 5, 7],
               [40, -1, 3, 5, 6, 8],
               [50, 3, 5, 7, 8, 9],
               [60, 6, 8, 9, 10, 10],
               [70, 10, 11, 11, 11, 11]],
    Smoking = [[0, 0],
               [1, 1]],
    Hist_Hyp = [[0, 0],
                [1, 1],
                [2, 2]],
    Risk_1yr = [[-12, 0.06],
                [-11, 0.07],
                [-10, 0.08],
                [-9, 0.09],
                [-8, 0.11],
                [-7, 0.13],
                [-6, 0.15],
                [-5, 0.18],
                [-4, 0.22],
                [-3, 0.26],
                [-2, 0.30],
                [-1, 0.36],
                [0,0.43],
                [1, 0.50],
                [2, 0.60],
                [3, 0.71],
                [4, 0.84],
                [5, 0.99],
                [6, 1.17],
                [7, 1.39],
                [8, 1.64],
                [9, 1.94],
                [10, 2.29],
                [11, 2.71],
                [12, 3.20],
                [13, 3.78],
                [14, 4.46],
                [15, 5.26],
                [16, 6.20],
                [17, 7.30],
                [18, 8.58],
                [19, 10.08],
                [20, 11.82],
                [21, 13.85],
                [22, 16.18],
                [23, 18.86],
                [24, 21.93],
                [25, 25.41],
                [26, 29.33],
                [27, 33.71],
                [28, 38.55]]
      )

CVDBetas_Female = [2.32888, 1.20904, -0.70833, 2.76157, 2.82263, 0.52873, 0.69154]
CVD_RiskNorm_Female = 26.1931
CVDBetas_Male = [3.06117, 1.12370, -0.93263, 1.93303, 1.99881, 0.65451, 0.57367]
CVD_RiskNorm_Male = 23.9802

EvalRisk() = results in a printout of risk of the patient developing hypertension/CVD in the future based on their current physiologic values reported on an appointment-by-appointment basis.
  
  CVD risk model: 
    
    D’Agostino, et al (2008). General Cardiovascular Risk Profile for Use in Primary Care. Circulation, 117(6), 743-753.

  Hypertension risk model:  
  
    Parikh, N. I., et al (2008). A Risk Score for Predicting Near-Term Incidence of Hypertension: The Framingham Heart Study. Annals of Internal Medicine, 148(2), 102. 


In [ ]:
def StoreBMI(PID): #Takes weight and height for patient and computes BMI
  for k in range(len(PatientDirectory['PID_' + PID]['Weight'])):
    kg = PatientDirectory['PID_' + PID]['Weight'][k]
    meter = PatientDirectory['PID_' + PID]['Height'][k]
    BMI = kg / (meter * meter)
    PatientDirectory['PID_' + PID]['BMI'][k] = BMI

def IdentifyPoints(PID, Appt, Category): #Searches through category to find the point value ascribed to the patient's value in that category
  Bounds = [i[0] for i in HypertensionRiskPoints[Category]]
  Value = PatientDirectory['PID_' + PID][Category][Appt]
  Closest = max(n for n in Bounds if n <= Value)
  Position = min(range(len(Bounds)), key = lambda i: abs(Bounds[i]-Closest)) 
  Points = HypertensionRiskPoints[Category][Position][1]
  return Points

def AgeDBPComposite(Age, DBP): #For hypertension outlook, takes the patient's values and finds the appropriate point value
  Column = [i[0] for i in HypertensionRiskPoints['Age_DBP']]
  Row = HypertensionRiskPoints['Age_DBP'][0]
  Closest_Age = max(n for n in Column if n <= Age)
  Closest_DBP = max(n for n in Row if n <= DBP)
  y = min(range(len(Column)), key = lambda i: abs(Column[i]-Closest_Age))
  x = min(range(len(Row)), key = lambda i: abs(Row[i]-Closest_DBP))
  P = HypertensionRiskPoints['Age_DBP'][y][x]
  return P

def GrabPercent(PID, Appt, Value): #Takes the point value computed from algorithm and finds the appropriate percent value
  Bounds = [i[0] for i in HypertensionRiskPoints['Risk_1yr']]
  Closest = max(n for n in Bounds if n <= Value)
  Position = min(range(len(Bounds)), key = lambda i: abs(Bounds[i]-Closest))  
  Percent = HypertensionRiskPoints['Risk_1yr'][Position][1]
  return Percent

def log_Cond(x): #function with computes the natural log of the input with the condition that if the input is 0 or 1 it returns the input as the output
  if x == 0:
    result = 0
  elif x == 1:
    result = 1
  else:
    n = 10000.0
    result = n * ((x **(1 / n)) - 1)
  return result

def CVDRiskEquation(PID, appt): #Computes the risk of the the patient developing cardiovascular disease in the next ten years based off of the current appointment values
  Age = PatientDirectory['PID_' + PID]['Age'][appt]
  Cholesterol = PatientDirectory['PID_' + PID]['Cholesterol'][appt]
  HDL = PatientDirectory['PID_' + PID]['HDL'][appt]
  SBP= PatientDirectory['PID_' + PID]['SBP'][appt]
  Smoker = PatientDirectory['PID_' + PID]['Smoking'][appt]
  Diabetic = PatientDirectory['PID_' + PID]['Diabetic'][appt]
  Risk_Unique = 0  
  if PatientDirectory['PID_' + PID]['Diag_Hyp'][appt] != 0: #Determines if the patient is being treated for hypertension
    PatientValues = [Age, Cholesterol, HDL, 0, SBP, Smoker, Diabetic]
  else:
    PatientValues = [Age, Cholesterol, HDL, SBP, 0, Smoker, Diabetic]

  for i in range(7): #Impact of patient values on the culmination of CVD risk
    if PatientDirectory['PID_' + PID]['Sex'][appt] == 0:
      Risk_Unique = Risk_Unique + ( CVDBetas_Female[i] * log_Cond(PatientValues[i]) )
      Percent = (1 - (0.95012 ** (2.7183**(Risk_Unique - CVD_RiskNorm_Female)))) * 100
    else:
      Risk_Unique = Risk_Unique + ( CVDBetas_Male[i] * log_Cond(PatientValues[i]) )
      Percent = (1 - (0.88936 ** (2.7183**(Risk_Unique - CVD_RiskNorm_Male)))) * 100  
  return Percent


def HypertensionRisk(PID): #Calls other functions which help to compute the patients risk of developing hypertension within the next year
  HypertensionKeyList = ['SBP', 'BMI', 'Sex', 'Smoking', 'Hist_Hyp']
  for i in range(len(PatientDirectory['PID_' + PID]['SBP'])): #iterate by appointment
      Points = 0
      for j in HypertensionKeyList: #iterate by category
        Points = Points + IdentifyPoints(PID, i, j)
      Points_AgeDBP = AgeDBPComposite( PatientDirectory['PID_' + PID]['Age'][i], PatientDirectory['PID_' + PID]['DBP'][i] )
      Points = Points + Points_AgeDBP
      Percent = GrabPercent(PID, i, Points)
      PatientDirectory['PID_' + PID]['Risk_Hyp'][i] = Percent

def CVDRisk(PID): #Calls other functions which help to compute the patients risk of developing CVD within the next 10 years
  for i in range(len(PatientDirectory['PID_' + PID]['SBP'])): #iterate by appointment
    Points = 0
    Percent = CVDRiskEquation(PID, i)
    PatientDirectory['PID_' + PID]['Risk_CVD'][i] = Percent     

def EvalRisk(PID):
  if PatientDirectory['PID_' + PID]['Diag_Hyp'][-1] == 1 or 0: #Pre/non hypertensive person
    HypertensionRisk(PID)

    
    Result = "             | Percent risk of developing \nDate (m/d/y) | hypertension within the next year"
    for i in range(len(PatientDirectory['PID_' + PID]['Date'])):
      
      dateline = PatientDirectory['PID_' + PID]['Date'][i]
      valueline = PatientDirectory['PID_' + PID]['Risk_Hyp'][i]
      Result = Result + "\n    %s | %02.2f %%" % (dateline, valueline)
       
  else:
    CVDRisk(PID)  
    
    Result = "             | Percent risk of developing Cardiovascular \nDate (m/d/y) | Disease within the next 10 years"
    for i in range(len(PatientDirectory['PID_' + PID]['Date'])):
      dateline = PatientDirectory['PID_' + PID]['Date'][i]
      valueline = PatientDirectory['PID_' + PID]['Risk_CVD'][i]
      Result = Result + "\n    %s | %02.2f %%" % (dateline, valueline)
  return Result

PID = "1" #input("Please Enter Patient ID: ")

StoreBMI(PID)

Result_EvalRisk = EvalRisk(PID)
print(Result_EvalRisk)

             | Percent risk of developing 
Date (m/d/y) | hypertension within the next year
    1/7/2020 | 6.20 %
    2/7/2020 | 10.08 %
    3/7/2020 | 11.82 %
